# Species

Assign species from organism or assay_organism columns or from assay descriptions.

In [1]:
from itertools import chain, combinations

import re

In [2]:
def insert_last_after(df, col):

    cols = df.columns.values.tolist()

    position = cols.index(col) + 1

    return df[cols[:position] + [cols[-1]] + cols[position:-1]]

In [3]:
# Load curve data (IC50, Ki, Kd etc)...

data = pd.read_pickle('data.pkl')

data.shape

(1317, 30)

In [4]:
description = data['description'].drop_duplicates()

description.size

135

In [5]:
queries = [
    {
          'name':  'Rabbit'
        , 'taxon': 'Oryctolagus cuniculus'
        , 'terms': ['rabbit']
    }
    , {
          'name':  'Guinea Pig'
        , 'taxon': 'Cavia porcellus'
        , 'terms': ['guinea pig']
    }
    , {
          'name': 'Pig'
        , 'taxon': 'Sus scrofa'
        , 'terms': ['pig', 'porcine']
    } , {
          'name':  'Cat'
        , 'taxon': 'Felis catus'
        , 'terms': ['cat', 'kitten']
    }    , {
          'name':  'Cow'
        , 'taxon': 'Bos taurus'
        , 'terms': ['cow', 'bovine', 'calf']
    }    , {
          'name':  'Rat'
        , 'taxon': 'Rattus norvegicus'
        , 'terms': ['rat']
    }    , {
          'name':  'Human'
        , 'taxon': 'Homo sapiens'
        , 'terms': ['human']
    }
]

for query in queries: query['pattern'] = r'\b(?:' + '|'.join(query['terms']) + r')\b'

In [6]:
def f(record):
    
    for query in queries:
        
        if record['organism'] == query['taxon'] or  record['assay_organism'] == query['taxon'] or re.search(query['pattern'], record['description'], re.I): 
            
            return query['name']
        
    return 'Other'

In [7]:
species = data[['organism', 'assay_organism', 'description']].drop_duplicates().reset_index(drop=True)

species[u'species2'] = species.apply(f, axis=1)

In [8]:
species.head()

,organism,assay_organism,description,species2
0,Homo sapiens,Felis catus,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Cat
1,Oryctolagus cuniculus,Oryctolagus cuniculus,Inhibition of [3H]nitrendipine binding to calcium channels in Rabbit cardiac muscle.,Rabbit
2,Rattus norvegicus,None,Calcium antagonistic activity by measuring [3H]nitrendipine displacement at L-type [Ca2+] channel in rat cortex homogenate,Rat
3,Rattus norvegicus,None,Inhibition of [3H]nitrendipine binding at L-type [Ca2+] channel in rat cortex homogenate by 50%.,Rat
4,Rattus norvegicus,Rattus norvegicus,Inhibition of [3H]D-888 binding to L-type [Ca2+] channel of membranes from rat skeletal muscle,Rat


In [9]:
species.to_pickle('species.pkl')

In [10]:
# Merge with main data table...

data = insert_last_after(data.merge(species), 'species')

HTML(data.head().to_html())

,target_chemblid,pref_name,target_type,organism,species,species2,tissue,relationship_type,assay_chemblid,description,assay_organism,parent_cmpd_chemblid,compound_class,standard_type,standard_relation,standard_value,standard_units,pchembl_value,activity_comment,data_validity_comment,potential_duplicate,cmpd_chemblid,compound_key,published_type,published_relation,published_value,published_units,doc_chemblid,pubmed_id,reference,active
0,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,260,nM,6.59,None,None,NaN,CHEMBL138302,2d,IC50,=,0.26,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1
1,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,390,nM,6.41,None,None,NaN,CHEMBL138302,2a,IC50,=,0.39,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1
2,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,500,nM,6.30,None,None,NaN,CHEMBL138302,2c,IC50,=,0.50,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1
3,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,270,nM,6.57,None,None,NaN,CHEMBL138302,2b,IC50,=,0.27,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1
4,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL343771,PAA,IC50,=,480,nM,6.32,None,None,NaN,CHEMBL343771,7a,IC50,=,0.48,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1


In [11]:
# Check there are no differences with the assignment performed via SQL...

data[data.species != data.species2].shape[0]

0